In [ ]:
%config Completer.use_jedi = False

# World Development Indicators

To supplement the disaster dataset, we have selected 55 different world development indicators from the [World Data Bank](https://databank.worldbank.org/source/world-development-indicators). We took the per-year data from year 2000 to year 2020 and for all available countries. Detailed indicator description is given in the metadata csv file `databank_wdi_metadata.csv`, including the source, unit of measure, periodicity, aggregation method, statistical concept and methodology, development relevance and limitations. Raw data is saved in `databank_wdi_data.csv`, with the preprocessed dataset created in this notebook saved in `databank_wdi_data_clean.csv`. Cheers.

In [ ]:
import numpy as np
import pandas as pd

pd.set_option("display.precision", 2)

In [ ]:
wdi = pd.read_csv("datasets/databank_wdi_data.csv", na_values="..")
wdi = wdi.rename(columns={
    "Country Name":"country_name",
    "Country Code":"country_code",
    "Series Name":"indicator_name",
    "Series Code":"indicator_code",
})
wdi = wdi.rename(columns={f"{y} [YR{y}]":f"{y}" for y in range(2000,2020+1)})
wdi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11935 entries, 0 to 11934
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country_name    11935 non-null  object 
 1   country_code    11935 non-null  object 
 2   indicator_name  11935 non-null  object 
 3   indicator_code  11935 non-null  object 
 4   2000            7691 non-null   float64
 5   2001            7552 non-null   float64
 6   2002            8042 non-null   float64
 7   2003            7853 non-null   float64
 8   2004            8113 non-null   float64
 9   2005            8230 non-null   float64
 10  2006            8253 non-null   float64
 11  2007            8668 non-null   float64
 12  2008            8303 non-null   float64
 13  2009            8369 non-null   float64
 14  2010            8444 non-null   float64
 15  2011            8447 non-null   float64
 16  2012            8868 non-null   float64
 17  2013            8436 non-null  

In [ ]:
wdi.describe()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
count,7.69e+03,7.55e+03,8.04e+03,7.85e+03,8.11e+03,8.23e+03,8.25e+03,8.67e+03,8.30e+03,8.37e+03,...,8.45e+03,8.87e+03,8.44e+03,8.55e+03,8.27e+03,8.42e+03,8.71e+03,8.26e+03,7.23e+03,4.18e+03
mean,1.54e+10,1.59e+10,1.54e+10,1.71e+10,1.84e+10,1.99e+10,2.17e+10,2.27e+10,2.54e+10,2.49e+10,...,2.88e+10,2.87e+10,3.16e+10,3.21e+10,3.27e+10,3.26e+10,3.30e+10,3.69e+10,4.37e+10,4.21e+10
std,2.41e+11,2.49e+11,2.48e+11,2.67e+11,2.86e+11,3.05e+11,3.24e+11,3.34e+11,3.56e+11,3.55e+11,...,4.01e+11,4.15e+11,4.44e+11,4.62e+11,4.80e+11,4.88e+11,5.05e+11,5.56e+11,6.24e+11,6.00e+11
min,-4.55e+09,-2.98e+09,-1.02e+09,-3.36e+09,-2.04e+10,-2.51e+10,-2.40e+09,-2.97e+10,-9.50e+09,-8.83e+09,...,-6.01e+09,-1.63e+10,-2.96e+10,-1.52e+10,-1.95e+10,-2.89e+10,-3.73e+10,-3.45e+11,-1.69e+11,-5.63e+01
25%,8.20e+00,7.99e+00,6.73e+00,8.75e+00,9.36e+00,9.16e+00,9.50e+00,8.07e+00,9.18e+00,8.27e+00,...,8.69e+00,7.10e+00,7.95e+00,7.87e+00,7.06e+00,6.38e+00,5.61e+00,6.23e+00,7.24e+00,3.27e+00
50%,6.30e+01,6.52e+01,6.19e+01,6.30e+01,6.47e+01,6.36e+01,6.35e+01,6.13e+01,6.44e+01,6.11e+01,...,6.44e+01,6.05e+01,6.39e+01,6.30e+01,5.59e+01,5.01e+01,4.74e+01,5.04e+01,5.83e+01,4.74e+01
75%,4.70e+03,6.01e+03,5.26e+03,4.77e+03,3.83e+03,3.21e+03,3.51e+03,3.31e+03,3.78e+03,3.45e+03,...,3.65e+03,3.20e+03,3.88e+03,3.66e+03,3.45e+03,1.83e+03,1.93e+03,2.52e+03,4.70e+03,1.26e+04
max,1.04e+13,1.07e+13,1.11e+13,1.15e+13,1.27e+13,1.36e+13,1.43e+13,1.46e+13,1.49e+13,1.46e+13,...,1.58e+13,1.67e+13,1.72e+13,1.81e+13,1.87e+13,1.90e+13,1.99e+13,2.16e+13,2.35e+13,2.13e+13


In [ ]:
print(f"Index unique: {wdi.index.is_unique}")
print(f"Dataframe has dupliates: {sum(wdi.duplicated()) > 0}, n={sum(wdi.duplicated())}")

Index unique: True
Dataframe has dupliates: False, n=0


In [ ]:
indicators = wdi.indicator_name.unique()
print(f"Number of indicators:{len(indicators)}")
print(f"Indicators:")
for i, s in enumerate(indicators):
    print(f"\t{i} --> {s}")

Number of indicators:55
Indicators:
	0 --> Adolescent fertility rate (births per 1,000 women ages 15-19)
	1 --> Agriculture, forestry, and fishing, value added (% of GDP)
	2 --> Annual freshwater withdrawals, total (% of internal resources)
	3 --> Births attended by skilled health staff (% of total)
	4 --> CO2 emissions (metric tons per capita)
	5 --> Contraceptive prevalence, any methods (% of women ages 15-49)
	6 --> Domestic credit provided by financial sector (% of GDP)
	7 --> Electric power consumption (kWh per capita)
	8 --> Energy use (kg of oil equivalent per capita)
	9 --> Exports of goods and services (% of GDP)
	10 --> External debt stocks, total (DOD, current US$)
	11 --> Fertility rate, total (births per woman)
	12 --> Foreign direct investment, net inflows (BoP, current US$)
	13 --> Forest area (sq. km)
	14 --> GDP (current US$)
	15 --> GDP growth (annual %)
	16 --> GNI per capita, Atlas method (current US$)
	17 --> GNI per capita, PPP (current international $)
	18 --> GN

In [ ]:
interesting_indicators = [
    "Population, total",
    "Population density (people per sq. km of land area)",
    "Surface area (sq. km)",
    "School enrollment, secondary (% gross)",
    "GDP (current US$)",
    "Energy use (kg of oil equivalent per capita)",
]

print(f"A list of some interesing indicators:")
for i, s in enumerate(indicators):
    if s in interesting_indicators:
        print(f"\t{i} --> {s}")

A list of some interesing indicators:
	8 --> Energy use (kg of oil equivalent per capita)
	14 --> GDP (current US$)
	36 --> Population density (people per sq. km of land area)
	38 --> Population, total
	47 --> School enrollment, secondary (% gross)
	49 --> Surface area (sq. km)


In [ ]:
croatia = wdi[wdi.country_name == "Croatia"]
# croatia
croatia[croatia.indicator_name.isin(interesting_indicators)]

,country_name,country_code,indicator_name,indicator_code,2000,2001,2002,2003,2004,2005,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
2648,Croatia,HRV,Energy use (kg of oil equivalent per capita),EG.USE.PCAP.KG.OE,1.88e+03,2.02e+03,2.08e+03,2.23e+03,2.23e+03,2.26e+03,...,2.14e+03,2.02e+03,1.98e+03,1.90e+03,NaN,NaN,NaN,NaN,NaN,NaN
2654,Croatia,HRV,GDP (current US$),NY.GDP.MKTP.CD,2.16e+10,2.31e+10,2.68e+10,3.47e+10,4.16e+10,4.54e+10,...,6.25e+10,5.66e+10,5.82e+10,5.76e+10,4.95e+10,5.16e+10,5.55e+10,6.14e+10,6.08e+10,5.60e+10
2676,Croatia,HRV,Population density (people per sq. km of land ...,EN.POP.DNST,7.99e+01,7.69e+01,7.69e+01,7.70e+01,7.69e+01,7.70e+01,...,7.65e+01,7.63e+01,7.60e+01,7.49e+01,7.43e+01,7.38e+01,7.29e+01,7.22e+01,7.18e+01,7.15e+01
2678,Croatia,HRV,"Population, total",SP.POP.TOTL,4.47e+06,4.30e+06,4.30e+06,4.30e+06,4.30e+06,4.31e+06,...,4.28e+06,4.27e+06,4.26e+06,4.24e+06,4.20e+06,4.17e+06,4.12e+06,4.09e+06,4.07e+06,4.05e+06
2687,Croatia,HRV,"School enrollment, secondary (% gross)",SE.SEC.ENRR,8.55e+01,8.58e+01,8.69e+01,8.83e+01,NaN,9.32e+01,...,9.92e+01,9.89e+01,9.93e+01,9.87e+01,9.87e+01,9.89e+01,9.94e+01,1.00e+02,1.00e+02,NaN
2689,Croatia,HRV,Surface area (sq. km),AG.SRF.TOTL.K2,5.65e+04,5.65e+04,5.65e+04,5.65e+04,5.66e+04,5.66e+04,...,5.66e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,NaN,NaN


In [ ]:
haiti = wdi[wdi.country_name == "Haiti"]
haiti[haiti.indicator_name.isin(interesting_indicators)]

,country_name,country_code,indicator_name,indicator_code,2000,2001,2002,2003,2004,2005,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
4628,Haiti,HTI,Energy use (kg of oil equivalent per capita),EG.USE.PCAP.KG.OE,2.38e+02,2.39e+02,2.64e+02,2.49e+02,2.53e+02,3.70e+02,...,3.89e+02,3.97e+02,3.94e+02,3.94e+02,NaN,NaN,NaN,NaN,NaN,NaN
4634,Haiti,HTI,GDP (current US$),NY.GDP.MKTP.CD,6.87e+09,6.36e+09,6.11e+09,5.00e+09,6.01e+09,7.05e+09,...,1.30e+10,1.37e+10,1.45e+10,1.48e+10,1.49e+10,1.37e+10,1.42e+10,1.60e+10,1.43e+10,1.34e+10
4656,Haiti,HTI,Population density (people per sq. km of land ...,EN.POP.DNST,3.07e+02,3.12e+02,3.18e+02,3.23e+02,3.28e+02,3.34e+02,...,3.66e+02,3.72e+02,3.77e+02,3.83e+02,3.88e+02,3.93e+02,3.98e+02,4.04e+02,4.09e+02,4.14e+02
4658,Haiti,HTI,"Population, total",SP.POP.TOTL,8.46e+06,8.61e+06,8.75e+06,8.90e+06,9.05e+06,9.20e+06,...,1.01e+07,1.03e+07,1.04e+07,1.05e+07,1.07e+07,1.08e+07,1.10e+07,1.11e+07,1.13e+07,1.14e+07
4667,Haiti,HTI,"School enrollment, secondary (% gross)",SE.SEC.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4669,Haiti,HTI,Surface area (sq. km),AG.SRF.TOTL.K2,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,...,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,NaN,NaN


## Discussion

Some data is missing. For some countries like Haiti, there was no information on `School enrollment, secondary (% gross)`, whereas for Croatia, there was no data about it only for year 2020. Missing data will need to be taken care of when applying machine learning models. Different models have different "preferences" which we will need to take care of ad hoc. It will likely make sense to interpolate some of the missing fields. For example, if the country surface area was reported for the past 3 years, but was not reported for year 2020 (like for Croatia), we might just want to copy the surface area from the previous year, as it very likely did not change. Or we could compute it by dividing the population by the population density. Overall, data polishing might require a lot of work and we should be cautious about it when reaching towards machine learning models.

TODO: Use small multiples to create tons of graphs
TODO: Make sure that there are no obvious outliers by plotting some boxplots
TODO: Merge with the disasters

In [ ]:
%config Completer.use_jedi = False

# World Development Indicators

To supplement the disaster dataset, we have selected 55 different world development indicators from the [World Data Bank](https://databank.worldbank.org/source/world-development-indicators). We took the per-year data from year 2000 to year 2020 and for all available countries. Detailed indicator description is given in the metadata csv file `databank_wdi_metadata.csv`, including the source, unit of measure, periodicity, aggregation method, statistical concept and methodology, development relevance and limitations. Raw data is saved in `databank_wdi_data.csv`, with the preprocessed dataset created in this notebook saved in `databank_wdi_data_clean.csv`. Cheers.

In [ ]:
import numpy as np
import pandas as pd

pd.set_option("display.precision", 2)

In [ ]:
wdi = pd.read_csv("datasets/databank_wdi_data.csv", na_values="..")
wdi = wdi.rename(columns={
    "Country Name":"country_name",
    "Country Code":"country_code",
    "Series Name":"indicator_name",
    "Series Code":"indicator_code",
})
wdi = wdi.rename(columns={f"{y} [YR{y}]":f"{y}" for y in range(2000,2020+1)})
wdi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11935 entries, 0 to 11934
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country_name    11935 non-null  object 
 1   country_code    11935 non-null  object 
 2   indicator_name  11935 non-null  object 
 3   indicator_code  11935 non-null  object 
 4   2000            7691 non-null   float64
 5   2001            7552 non-null   float64
 6   2002            8042 non-null   float64
 7   2003            7853 non-null   float64
 8   2004            8113 non-null   float64
 9   2005            8230 non-null   float64
 10  2006            8253 non-null   float64
 11  2007            8668 non-null   float64
 12  2008            8303 non-null   float64
 13  2009            8369 non-null   float64
 14  2010            8444 non-null   float64
 15  2011            8447 non-null   float64
 16  2012            8868 non-null   float64
 17  2013            8436 non-null  

In [ ]:
wdi.describe()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
count,7.69e+03,7.55e+03,8.04e+03,7.85e+03,8.11e+03,8.23e+03,8.25e+03,8.67e+03,8.30e+03,8.37e+03,...,8.45e+03,8.87e+03,8.44e+03,8.55e+03,8.27e+03,8.42e+03,8.71e+03,8.26e+03,7.23e+03,4.18e+03
mean,1.54e+10,1.59e+10,1.54e+10,1.71e+10,1.84e+10,1.99e+10,2.17e+10,2.27e+10,2.54e+10,2.49e+10,...,2.88e+10,2.87e+10,3.16e+10,3.21e+10,3.27e+10,3.26e+10,3.30e+10,3.69e+10,4.37e+10,4.21e+10
std,2.41e+11,2.49e+11,2.48e+11,2.67e+11,2.86e+11,3.05e+11,3.24e+11,3.34e+11,3.56e+11,3.55e+11,...,4.01e+11,4.15e+11,4.44e+11,4.62e+11,4.80e+11,4.88e+11,5.05e+11,5.56e+11,6.24e+11,6.00e+11
min,-4.55e+09,-2.98e+09,-1.02e+09,-3.36e+09,-2.04e+10,-2.51e+10,-2.40e+09,-2.97e+10,-9.50e+09,-8.83e+09,...,-6.01e+09,-1.63e+10,-2.96e+10,-1.52e+10,-1.95e+10,-2.89e+10,-3.73e+10,-3.45e+11,-1.69e+11,-5.63e+01
25%,8.20e+00,7.99e+00,6.73e+00,8.75e+00,9.36e+00,9.16e+00,9.50e+00,8.07e+00,9.18e+00,8.27e+00,...,8.69e+00,7.10e+00,7.95e+00,7.87e+00,7.06e+00,6.38e+00,5.61e+00,6.23e+00,7.24e+00,3.27e+00
50%,6.30e+01,6.52e+01,6.19e+01,6.30e+01,6.47e+01,6.36e+01,6.35e+01,6.13e+01,6.44e+01,6.11e+01,...,6.44e+01,6.05e+01,6.39e+01,6.30e+01,5.59e+01,5.01e+01,4.74e+01,5.04e+01,5.83e+01,4.74e+01
75%,4.70e+03,6.01e+03,5.26e+03,4.77e+03,3.83e+03,3.21e+03,3.51e+03,3.31e+03,3.78e+03,3.45e+03,...,3.65e+03,3.20e+03,3.88e+03,3.66e+03,3.45e+03,1.83e+03,1.93e+03,2.52e+03,4.70e+03,1.26e+04
max,1.04e+13,1.07e+13,1.11e+13,1.15e+13,1.27e+13,1.36e+13,1.43e+13,1.46e+13,1.49e+13,1.46e+13,...,1.58e+13,1.67e+13,1.72e+13,1.81e+13,1.87e+13,1.90e+13,1.99e+13,2.16e+13,2.35e+13,2.13e+13


In [ ]:
print(f"Index unique: {wdi.index.is_unique}")
print(f"Dataframe has dupliates: {sum(wdi.duplicated()) > 0}, n={sum(wdi.duplicated())}")

Index unique: True
Dataframe has dupliates: False, n=0


In [ ]:
indicators = wdi.indicator_name.unique()
print(f"Number of indicators:{len(indicators)}")
print(f"Indicators:")
for i, s in enumerate(indicators):
    print(f"\t{i} --> {s}")

Number of indicators:55
Indicators:
	0 --> Adolescent fertility rate (births per 1,000 women ages 15-19)
	1 --> Agriculture, forestry, and fishing, value added (% of GDP)
	2 --> Annual freshwater withdrawals, total (% of internal resources)
	3 --> Births attended by skilled health staff (% of total)
	4 --> CO2 emissions (metric tons per capita)
	5 --> Contraceptive prevalence, any methods (% of women ages 15-49)
	6 --> Domestic credit provided by financial sector (% of GDP)
	7 --> Electric power consumption (kWh per capita)
	8 --> Energy use (kg of oil equivalent per capita)
	9 --> Exports of goods and services (% of GDP)
	10 --> External debt stocks, total (DOD, current US$)
	11 --> Fertility rate, total (births per woman)
	12 --> Foreign direct investment, net inflows (BoP, current US$)
	13 --> Forest area (sq. km)
	14 --> GDP (current US$)
	15 --> GDP growth (annual %)
	16 --> GNI per capita, Atlas method (current US$)
	17 --> GNI per capita, PPP (current international $)
	18 --> GN

In [ ]:
interesting_indicators = [
    "Population, total",
    "Population density (people per sq. km of land area)",
    "Surface area (sq. km)",
    "School enrollment, secondary (% gross)",
    "GDP (current US$)",
    "Energy use (kg of oil equivalent per capita)",
]

print(f"A list of some interesing indicators:")
for i, s in enumerate(indicators):
    if s in interesting_indicators:
        print(f"\t{i} --> {s}")

A list of some interesing indicators:
	8 --> Energy use (kg of oil equivalent per capita)
	14 --> GDP (current US$)
	36 --> Population density (people per sq. km of land area)
	38 --> Population, total
	47 --> School enrollment, secondary (% gross)
	49 --> Surface area (sq. km)


In [ ]:
croatia = wdi[wdi.country_name == "Croatia"]
# croatia
croatia[croatia.indicator_name.isin(interesting_indicators)]

,country_name,country_code,indicator_name,indicator_code,2000,2001,2002,2003,2004,2005,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
2648,Croatia,HRV,Energy use (kg of oil equivalent per capita),EG.USE.PCAP.KG.OE,1.88e+03,2.02e+03,2.08e+03,2.23e+03,2.23e+03,2.26e+03,...,2.14e+03,2.02e+03,1.98e+03,1.90e+03,NaN,NaN,NaN,NaN,NaN,NaN
2654,Croatia,HRV,GDP (current US$),NY.GDP.MKTP.CD,2.16e+10,2.31e+10,2.68e+10,3.47e+10,4.16e+10,4.54e+10,...,6.25e+10,5.66e+10,5.82e+10,5.76e+10,4.95e+10,5.16e+10,5.55e+10,6.14e+10,6.08e+10,5.60e+10
2676,Croatia,HRV,Population density (people per sq. km of land ...,EN.POP.DNST,7.99e+01,7.69e+01,7.69e+01,7.70e+01,7.69e+01,7.70e+01,...,7.65e+01,7.63e+01,7.60e+01,7.49e+01,7.43e+01,7.38e+01,7.29e+01,7.22e+01,7.18e+01,7.15e+01
2678,Croatia,HRV,"Population, total",SP.POP.TOTL,4.47e+06,4.30e+06,4.30e+06,4.30e+06,4.30e+06,4.31e+06,...,4.28e+06,4.27e+06,4.26e+06,4.24e+06,4.20e+06,4.17e+06,4.12e+06,4.09e+06,4.07e+06,4.05e+06
2687,Croatia,HRV,"School enrollment, secondary (% gross)",SE.SEC.ENRR,8.55e+01,8.58e+01,8.69e+01,8.83e+01,NaN,9.32e+01,...,9.92e+01,9.89e+01,9.93e+01,9.87e+01,9.87e+01,9.89e+01,9.94e+01,1.00e+02,1.00e+02,NaN
2689,Croatia,HRV,Surface area (sq. km),AG.SRF.TOTL.K2,5.65e+04,5.65e+04,5.65e+04,5.65e+04,5.66e+04,5.66e+04,...,5.66e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,NaN,NaN


In [ ]:
haiti = wdi[wdi.country_name == "Haiti"]
haiti[haiti.indicator_name.isin(interesting_indicators)]

,country_name,country_code,indicator_name,indicator_code,2000,2001,2002,2003,2004,2005,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
4628,Haiti,HTI,Energy use (kg of oil equivalent per capita),EG.USE.PCAP.KG.OE,2.38e+02,2.39e+02,2.64e+02,2.49e+02,2.53e+02,3.70e+02,...,3.89e+02,3.97e+02,3.94e+02,3.94e+02,NaN,NaN,NaN,NaN,NaN,NaN
4634,Haiti,HTI,GDP (current US$),NY.GDP.MKTP.CD,6.87e+09,6.36e+09,6.11e+09,5.00e+09,6.01e+09,7.05e+09,...,1.30e+10,1.37e+10,1.45e+10,1.48e+10,1.49e+10,1.37e+10,1.42e+10,1.60e+10,1.43e+10,1.34e+10
4656,Haiti,HTI,Population density (people per sq. km of land ...,EN.POP.DNST,3.07e+02,3.12e+02,3.18e+02,3.23e+02,3.28e+02,3.34e+02,...,3.66e+02,3.72e+02,3.77e+02,3.83e+02,3.88e+02,3.93e+02,3.98e+02,4.04e+02,4.09e+02,4.14e+02
4658,Haiti,HTI,"Population, total",SP.POP.TOTL,8.46e+06,8.61e+06,8.75e+06,8.90e+06,9.05e+06,9.20e+06,...,1.01e+07,1.03e+07,1.04e+07,1.05e+07,1.07e+07,1.08e+07,1.10e+07,1.11e+07,1.13e+07,1.14e+07
4667,Haiti,HTI,"School enrollment, secondary (% gross)",SE.SEC.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4669,Haiti,HTI,Surface area (sq. km),AG.SRF.TOTL.K2,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,...,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,NaN,NaN


In [ ]:
%config Completer.use_jedi = False

# World Development Indicators

To supplement the disaster dataset, we have selected 55 different world development indicators from the [World Data Bank](https://databank.worldbank.org/source/world-development-indicators). We took the per-year data from year 2000 to year 202 and for all available countries. Detailed indicator description is given in the metadata csv file `databank_wdi_metadata.csv`, including the source, unit of measure, periodicity, aggregation method, statistical concept and methodology, development relevance and limitations. Raw data is saved in `databank_wdi_data.csv`, with the preprocessed dataset created in this notebook saved in `databank_wdi_data_clean.csv`. Cheers.

In [ ]:
import numpy as np
import pandas as pd

pd.set_option("display.precision", 2)

In [ ]:
wdi = pd.read_csv("datasets/databank_wdi_data.csv", na_values="..")
wdi = wdi.rename(columns={
    "Country Name":"country_name",
    "Country Code":"country_code",
    "Series Name":"indicator_name",
    "Series Code":"indicator_code",
})
wdi = wdi.rename(columns={f"{y} [YR{y}]":f"{y}" for y in range(2000,2020+1)})
wdi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11935 entries, 0 to 11934
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country_name    11935 non-null  object 
 1   country_code    11935 non-null  object 
 2   indicator_name  11935 non-null  object 
 3   indicator_code  11935 non-null  object 
 4   2000            7691 non-null   float64
 5   2001            7552 non-null   float64
 6   2002            8042 non-null   float64
 7   2003            7853 non-null   float64
 8   2004            8113 non-null   float64
 9   2005            8230 non-null   float64
 10  2006            8253 non-null   float64
 11  2007            8668 non-null   float64
 12  2008            8303 non-null   float64
 13  2009            8369 non-null   float64
 14  2010            8444 non-null   float64
 15  2011            8447 non-null   float64
 16  2012            8868 non-null   float64
 17  2013            8436 non-null  

In [ ]:
wdi.describe()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
count,7.69e+03,7.55e+03,8.04e+03,7.85e+03,8.11e+03,8.23e+03,8.25e+03,8.67e+03,8.30e+03,8.37e+03,...,8.45e+03,8.87e+03,8.44e+03,8.55e+03,8.27e+03,8.42e+03,8.71e+03,8.26e+03,7.23e+03,4.18e+03
mean,1.54e+10,1.59e+10,1.54e+10,1.71e+10,1.84e+10,1.99e+10,2.17e+10,2.27e+10,2.54e+10,2.49e+10,...,2.88e+10,2.87e+10,3.16e+10,3.21e+10,3.27e+10,3.26e+10,3.30e+10,3.69e+10,4.37e+10,4.21e+10
std,2.41e+11,2.49e+11,2.48e+11,2.67e+11,2.86e+11,3.05e+11,3.24e+11,3.34e+11,3.56e+11,3.55e+11,...,4.01e+11,4.15e+11,4.44e+11,4.62e+11,4.80e+11,4.88e+11,5.05e+11,5.56e+11,6.24e+11,6.00e+11
min,-4.55e+09,-2.98e+09,-1.02e+09,-3.36e+09,-2.04e+10,-2.51e+10,-2.40e+09,-2.97e+10,-9.50e+09,-8.83e+09,...,-6.01e+09,-1.63e+10,-2.96e+10,-1.52e+10,-1.95e+10,-2.89e+10,-3.73e+10,-3.45e+11,-1.69e+11,-5.63e+01
25%,8.20e+00,7.99e+00,6.73e+00,8.75e+00,9.36e+00,9.16e+00,9.50e+00,8.07e+00,9.18e+00,8.27e+00,...,8.69e+00,7.10e+00,7.95e+00,7.87e+00,7.06e+00,6.38e+00,5.61e+00,6.23e+00,7.24e+00,3.27e+00
50%,6.30e+01,6.52e+01,6.19e+01,6.30e+01,6.47e+01,6.36e+01,6.35e+01,6.13e+01,6.44e+01,6.11e+01,...,6.44e+01,6.05e+01,6.39e+01,6.30e+01,5.59e+01,5.01e+01,4.74e+01,5.04e+01,5.83e+01,4.74e+01
75%,4.70e+03,6.01e+03,5.26e+03,4.77e+03,3.83e+03,3.21e+03,3.51e+03,3.31e+03,3.78e+03,3.45e+03,...,3.65e+03,3.20e+03,3.88e+03,3.66e+03,3.45e+03,1.83e+03,1.93e+03,2.52e+03,4.70e+03,1.26e+04
max,1.04e+13,1.07e+13,1.11e+13,1.15e+13,1.27e+13,1.36e+13,1.43e+13,1.46e+13,1.49e+13,1.46e+13,...,1.58e+13,1.67e+13,1.72e+13,1.81e+13,1.87e+13,1.90e+13,1.99e+13,2.16e+13,2.35e+13,2.13e+13


In [ ]:
print(f"Index unique: {wdi.index.is_unique}")
print(f"Dataframe has dupliates: {sum(wdi.duplicated()) > 0}, n={sum(wdi.duplicated())}")

Index unique: True
Dataframe has dupliates: False, n=0


In [ ]:
indicators = wdi.indicator_name.unique()
print(f"Number of indicators:{len(indicators)}")
print(f"Indicators:")
for i, s in enumerate(indicators):
    print(f"\t{i} --> {s}")

Number of indicators:55
Indicators:
	0 --> Adolescent fertility rate (births per 1,000 women ages 15-19)
	1 --> Agriculture, forestry, and fishing, value added (% of GDP)
	2 --> Annual freshwater withdrawals, total (% of internal resources)
	3 --> Births attended by skilled health staff (% of total)
	4 --> CO2 emissions (metric tons per capita)
	5 --> Contraceptive prevalence, any methods (% of women ages 15-49)
	6 --> Domestic credit provided by financial sector (% of GDP)
	7 --> Electric power consumption (kWh per capita)
	8 --> Energy use (kg of oil equivalent per capita)
	9 --> Exports of goods and services (% of GDP)
	10 --> External debt stocks, total (DOD, current US$)
	11 --> Fertility rate, total (births per woman)
	12 --> Foreign direct investment, net inflows (BoP, current US$)
	13 --> Forest area (sq. km)
	14 --> GDP (current US$)
	15 --> GDP growth (annual %)
	16 --> GNI per capita, Atlas method (current US$)
	17 --> GNI per capita, PPP (current international $)
	18 --> GN

In [ ]:
interesting_indicators = [
    "Population, total",
    "Population density (people per sq. km of land area)",
    "Surface area (sq. km)",
    "School enrollment, secondary (% gross)",
    "GDP (current US$)",
    "Energy use (kg of oil equivalent per capita)",
]

print(f"A list of some interesing indicators:")
for i, s in enumerate(indicators):
    if s in interesting_indicators:
        print(f"\t{i} --> {s}")

A list of some interesing indicators:
	8 --> Energy use (kg of oil equivalent per capita)
	14 --> GDP (current US$)
	36 --> Population density (people per sq. km of land area)
	38 --> Population, total
	47 --> School enrollment, secondary (% gross)
	49 --> Surface area (sq. km)


In [ ]:
croatia = wdi[wdi.country_name == "Croatia"]
# croatia
croatia[croatia.indicator_name.isin(interesting_indicators)]

,country_name,country_code,indicator_name,indicator_code,2000,2001,2002,2003,2004,2005,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
2648,Croatia,HRV,Energy use (kg of oil equivalent per capita),EG.USE.PCAP.KG.OE,1.88e+03,2.02e+03,2.08e+03,2.23e+03,2.23e+03,2.26e+03,...,2.14e+03,2.02e+03,1.98e+03,1.90e+03,NaN,NaN,NaN,NaN,NaN,NaN
2654,Croatia,HRV,GDP (current US$),NY.GDP.MKTP.CD,2.16e+10,2.31e+10,2.68e+10,3.47e+10,4.16e+10,4.54e+10,...,6.25e+10,5.66e+10,5.82e+10,5.76e+10,4.95e+10,5.16e+10,5.55e+10,6.14e+10,6.08e+10,5.60e+10
2676,Croatia,HRV,Population density (people per sq. km of land ...,EN.POP.DNST,7.99e+01,7.69e+01,7.69e+01,7.70e+01,7.69e+01,7.70e+01,...,7.65e+01,7.63e+01,7.60e+01,7.49e+01,7.43e+01,7.38e+01,7.29e+01,7.22e+01,7.18e+01,7.15e+01
2678,Croatia,HRV,"Population, total",SP.POP.TOTL,4.47e+06,4.30e+06,4.30e+06,4.30e+06,4.30e+06,4.31e+06,...,4.28e+06,4.27e+06,4.26e+06,4.24e+06,4.20e+06,4.17e+06,4.12e+06,4.09e+06,4.07e+06,4.05e+06
2687,Croatia,HRV,"School enrollment, secondary (% gross)",SE.SEC.ENRR,8.55e+01,8.58e+01,8.69e+01,8.83e+01,NaN,9.32e+01,...,9.92e+01,9.89e+01,9.93e+01,9.87e+01,9.87e+01,9.89e+01,9.94e+01,1.00e+02,1.00e+02,NaN
2689,Croatia,HRV,Surface area (sq. km),AG.SRF.TOTL.K2,5.65e+04,5.65e+04,5.65e+04,5.65e+04,5.66e+04,5.66e+04,...,5.66e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,NaN,NaN


In [ ]:
haiti = wdi[wdi.country_name == "Haiti"]
haiti[haiti.indicator_name.isin(interesting_indicators)]

,country_name,country_code,indicator_name,indicator_code,2000,2001,2002,2003,2004,2005,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
4628,Haiti,HTI,Energy use (kg of oil equivalent per capita),EG.USE.PCAP.KG.OE,2.38e+02,2.39e+02,2.64e+02,2.49e+02,2.53e+02,3.70e+02,...,3.89e+02,3.97e+02,3.94e+02,3.94e+02,NaN,NaN,NaN,NaN,NaN,NaN
4634,Haiti,HTI,GDP (current US$),NY.GDP.MKTP.CD,6.87e+09,6.36e+09,6.11e+09,5.00e+09,6.01e+09,7.05e+09,...,1.30e+10,1.37e+10,1.45e+10,1.48e+10,1.49e+10,1.37e+10,1.42e+10,1.60e+10,1.43e+10,1.34e+10
4656,Haiti,HTI,Population density (people per sq. km of land ...,EN.POP.DNST,3.07e+02,3.12e+02,3.18e+02,3.23e+02,3.28e+02,3.34e+02,...,3.66e+02,3.72e+02,3.77e+02,3.83e+02,3.88e+02,3.93e+02,3.98e+02,4.04e+02,4.09e+02,4.14e+02
4658,Haiti,HTI,"Population, total",SP.POP.TOTL,8.46e+06,8.61e+06,8.75e+06,8.90e+06,9.05e+06,9.20e+06,...,1.01e+07,1.03e+07,1.04e+07,1.05e+07,1.07e+07,1.08e+07,1.10e+07,1.11e+07,1.13e+07,1.14e+07
4667,Haiti,HTI,"School enrollment, secondary (% gross)",SE.SEC.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4669,Haiti,HTI,Surface area (sq. km),AG.SRF.TOTL.K2,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,...,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,NaN,NaN


In [ ]:
%config Completer.use_jedi = False

# World Development Indicators

To supplement the disaster dataset, we have selected 55 different world development indicators from the [World Data Bank](https://databank.worldbank.org/source/world-development-indicators). We took the per-year data from year 2000 to year 2020 and for all available countries. Detailed indicator description is given in the metadata csv file `databank_wdi_metadata.csv`, including the source, unit of measure, periodicity, aggregation method, statistical concept and methodology, development relevance and limitations. Raw data is saved in `databank_wdi_data.csv`, with the preprocessed dataset created in this notebook saved in `databank_wdi_data_clean.csv`. Cheers.

In [ ]:
import numpy as np
import pandas as pd

pd.set_option("display.precision", 2)

In [ ]:
wdi = pd.read_csv("datasets/databank_wdi_data.csv", na_values="..")
wdi = wdi.rename(columns={
    "Country Name":"country_name",
    "Country Code":"country_code",
    "Series Name":"indicator_name",
    "Series Code":"indicator_code",
})
wdi = wdi.rename(columns={f"{y} [YR{y}]":f"{y}" for y in range(2000,2020+1)})
wdi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11935 entries, 0 to 11934
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country_name    11935 non-null  object 
 1   country_code    11935 non-null  object 
 2   indicator_name  11935 non-null  object 
 3   indicator_code  11935 non-null  object 
 4   2000            7691 non-null   float64
 5   2001            7552 non-null   float64
 6   2002            8042 non-null   float64
 7   2003            7853 non-null   float64
 8   2004            8113 non-null   float64
 9   2005            8230 non-null   float64
 10  2006            8253 non-null   float64
 11  2007            8668 non-null   float64
 12  2008            8303 non-null   float64
 13  2009            8369 non-null   float64
 14  2010            8444 non-null   float64
 15  2011            8447 non-null   float64
 16  2012            8868 non-null   float64
 17  2013            8436 non-null  

In [ ]:
wdi.describe()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
count,7.69e+03,7.55e+03,8.04e+03,7.85e+03,8.11e+03,8.23e+03,8.25e+03,8.67e+03,8.30e+03,8.37e+03,...,8.45e+03,8.87e+03,8.44e+03,8.55e+03,8.27e+03,8.42e+03,8.71e+03,8.26e+03,7.23e+03,4.18e+03
mean,1.54e+10,1.59e+10,1.54e+10,1.71e+10,1.84e+10,1.99e+10,2.17e+10,2.27e+10,2.54e+10,2.49e+10,...,2.88e+10,2.87e+10,3.16e+10,3.21e+10,3.27e+10,3.26e+10,3.30e+10,3.69e+10,4.37e+10,4.21e+10
std,2.41e+11,2.49e+11,2.48e+11,2.67e+11,2.86e+11,3.05e+11,3.24e+11,3.34e+11,3.56e+11,3.55e+11,...,4.01e+11,4.15e+11,4.44e+11,4.62e+11,4.80e+11,4.88e+11,5.05e+11,5.56e+11,6.24e+11,6.00e+11
min,-4.55e+09,-2.98e+09,-1.02e+09,-3.36e+09,-2.04e+10,-2.51e+10,-2.40e+09,-2.97e+10,-9.50e+09,-8.83e+09,...,-6.01e+09,-1.63e+10,-2.96e+10,-1.52e+10,-1.95e+10,-2.89e+10,-3.73e+10,-3.45e+11,-1.69e+11,-5.63e+01
25%,8.20e+00,7.99e+00,6.73e+00,8.75e+00,9.36e+00,9.16e+00,9.50e+00,8.07e+00,9.18e+00,8.27e+00,...,8.69e+00,7.10e+00,7.95e+00,7.87e+00,7.06e+00,6.38e+00,5.61e+00,6.23e+00,7.24e+00,3.27e+00
50%,6.30e+01,6.52e+01,6.19e+01,6.30e+01,6.47e+01,6.36e+01,6.35e+01,6.13e+01,6.44e+01,6.11e+01,...,6.44e+01,6.05e+01,6.39e+01,6.30e+01,5.59e+01,5.01e+01,4.74e+01,5.04e+01,5.83e+01,4.74e+01
75%,4.70e+03,6.01e+03,5.26e+03,4.77e+03,3.83e+03,3.21e+03,3.51e+03,3.31e+03,3.78e+03,3.45e+03,...,3.65e+03,3.20e+03,3.88e+03,3.66e+03,3.45e+03,1.83e+03,1.93e+03,2.52e+03,4.70e+03,1.26e+04
max,1.04e+13,1.07e+13,1.11e+13,1.15e+13,1.27e+13,1.36e+13,1.43e+13,1.46e+13,1.49e+13,1.46e+13,...,1.58e+13,1.67e+13,1.72e+13,1.81e+13,1.87e+13,1.90e+13,1.99e+13,2.16e+13,2.35e+13,2.13e+13


In [ ]:
print(f"Index unique: {wdi.index.is_unique}")
print(f"Dataframe has dupliates: {sum(wdi.duplicated()) > 0}, n={sum(wdi.duplicated())}")

Index unique: True
Dataframe has dupliates: False, n=0


In [ ]:
indicators = wdi.indicator_name.unique()
print(f"Number of indicators:{len(indicators)}")
print(f"Indicators:")
for i, s in enumerate(indicators):
    print(f"\t{i} --> {s}")

Number of indicators:55
Indicators:
	0 --> Adolescent fertility rate (births per 1,000 women ages 15-19)
	1 --> Agriculture, forestry, and fishing, value added (% of GDP)
	2 --> Annual freshwater withdrawals, total (% of internal resources)
	3 --> Births attended by skilled health staff (% of total)
	4 --> CO2 emissions (metric tons per capita)
	5 --> Contraceptive prevalence, any methods (% of women ages 15-49)
	6 --> Domestic credit provided by financial sector (% of GDP)
	7 --> Electric power consumption (kWh per capita)
	8 --> Energy use (kg of oil equivalent per capita)
	9 --> Exports of goods and services (% of GDP)
	10 --> External debt stocks, total (DOD, current US$)
	11 --> Fertility rate, total (births per woman)
	12 --> Foreign direct investment, net inflows (BoP, current US$)
	13 --> Forest area (sq. km)
	14 --> GDP (current US$)
	15 --> GDP growth (annual %)
	16 --> GNI per capita, Atlas method (current US$)
	17 --> GNI per capita, PPP (current international $)
	18 --> GN

In [ ]:
interesting_indicators = [
    "Population, total",
    "Population density (people per sq. km of land area)",
    "Surface area (sq. km)",
    "School enrollment, secondary (% gross)",
    "GDP (current US$)",
    "Energy use (kg of oil equivalent per capita)",
]

print(f"A list of some interesing indicators:")
for i, s in enumerate(indicators):
    if s in interesting_indicators:
        print(f"\t{i} --> {s}")

A list of some interesing indicators:
	8 --> Energy use (kg of oil equivalent per capita)
	14 --> GDP (current US$)
	36 --> Population density (people per sq. km of land area)
	38 --> Population, total
	47 --> School enrollment, secondary (% gross)
	49 --> Surface area (sq. km)


In [ ]:
croatia = wdi[wdi.country_name == "Croatia"]
# croatia
croatia[croatia.indicator_name.isin(interesting_indicators)]

,country_name,country_code,indicator_name,indicator_code,2000,2001,2002,2003,2004,2005,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
2648,Croatia,HRV,Energy use (kg of oil equivalent per capita),EG.USE.PCAP.KG.OE,1.88e+03,2.02e+03,2.08e+03,2.23e+03,2.23e+03,2.26e+03,...,2.14e+03,2.02e+03,1.98e+03,1.90e+03,NaN,NaN,NaN,NaN,NaN,NaN
2654,Croatia,HRV,GDP (current US$),NY.GDP.MKTP.CD,2.16e+10,2.31e+10,2.68e+10,3.47e+10,4.16e+10,4.54e+10,...,6.25e+10,5.66e+10,5.82e+10,5.76e+10,4.95e+10,5.16e+10,5.55e+10,6.14e+10,6.08e+10,5.60e+10
2676,Croatia,HRV,Population density (people per sq. km of land ...,EN.POP.DNST,7.99e+01,7.69e+01,7.69e+01,7.70e+01,7.69e+01,7.70e+01,...,7.65e+01,7.63e+01,7.60e+01,7.49e+01,7.43e+01,7.38e+01,7.29e+01,7.22e+01,7.18e+01,7.15e+01
2678,Croatia,HRV,"Population, total",SP.POP.TOTL,4.47e+06,4.30e+06,4.30e+06,4.30e+06,4.30e+06,4.31e+06,...,4.28e+06,4.27e+06,4.26e+06,4.24e+06,4.20e+06,4.17e+06,4.12e+06,4.09e+06,4.07e+06,4.05e+06
2687,Croatia,HRV,"School enrollment, secondary (% gross)",SE.SEC.ENRR,8.55e+01,8.58e+01,8.69e+01,8.83e+01,NaN,9.32e+01,...,9.92e+01,9.89e+01,9.93e+01,9.87e+01,9.87e+01,9.89e+01,9.94e+01,1.00e+02,1.00e+02,NaN
2689,Croatia,HRV,Surface area (sq. km),AG.SRF.TOTL.K2,5.65e+04,5.65e+04,5.65e+04,5.65e+04,5.66e+04,5.66e+04,...,5.66e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,8.81e+04,NaN,NaN


In [ ]:
haiti = wdi[wdi.country_name == "Haiti"]
haiti[haiti.indicator_name.isin(interesting_indicators)]

,country_name,country_code,indicator_name,indicator_code,2000,2001,2002,2003,2004,2005,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
4628,Haiti,HTI,Energy use (kg of oil equivalent per capita),EG.USE.PCAP.KG.OE,2.38e+02,2.39e+02,2.64e+02,2.49e+02,2.53e+02,3.70e+02,...,3.89e+02,3.97e+02,3.94e+02,3.94e+02,NaN,NaN,NaN,NaN,NaN,NaN
4634,Haiti,HTI,GDP (current US$),NY.GDP.MKTP.CD,6.87e+09,6.36e+09,6.11e+09,5.00e+09,6.01e+09,7.05e+09,...,1.30e+10,1.37e+10,1.45e+10,1.48e+10,1.49e+10,1.37e+10,1.42e+10,1.60e+10,1.43e+10,1.34e+10
4656,Haiti,HTI,Population density (people per sq. km of land ...,EN.POP.DNST,3.07e+02,3.12e+02,3.18e+02,3.23e+02,3.28e+02,3.34e+02,...,3.66e+02,3.72e+02,3.77e+02,3.83e+02,3.88e+02,3.93e+02,3.98e+02,4.04e+02,4.09e+02,4.14e+02
4658,Haiti,HTI,"Population, total",SP.POP.TOTL,8.46e+06,8.61e+06,8.75e+06,8.90e+06,9.05e+06,9.20e+06,...,1.01e+07,1.03e+07,1.04e+07,1.05e+07,1.07e+07,1.08e+07,1.10e+07,1.11e+07,1.13e+07,1.14e+07
4667,Haiti,HTI,"School enrollment, secondary (% gross)",SE.SEC.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4669,Haiti,HTI,Surface area (sq. km),AG.SRF.TOTL.K2,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,...,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,2.78e+04,NaN,NaN


In [ ]:
%config Completer.use_jedi = False

In [ ]:
!pip install autoviz
from autoviz.AutoViz_Class import AutoViz_Class
AV = AutoViz_Class()
dft = AV.AutoViz(
    wdi_dataset,
    # ",",
    # "type",
    # header=0,
    # verbose=0,
    # lowess=False,
    # chart_format="svg",
    # max_rows_analyzed=1500000,
    # max_cols_analyzed=300,
)

In [ ]:
wdi_dataset.to_csv("datasets/databank_wdi_data_clean.csv")

## Discussion

Some data is missing. For some countries like Haiti, there was no information on `School enrollment, secondary (% gross)`, whereas for Croatia, there was no data about it only for year 2020. Missing data will need to be taken care of when applying machine learning models. Different models have different "preferences" which we will need to take care of ad hoc. It will likely make sense to interpolate some of the missing fields. For example, if the country surface area was reported for the past 3 years, but was not reported for year 2020 (like for Croatia), we might just want to copy the surface area from the previous year, as it very likely did not change. Or we could compute it by dividing the population by the population density. Overall, data polishing might require a lot of work and we should be cautious about it when reaching towards machine learning models.

TODO: Use small multiples to create tons of graphs
TODO: Make sure that there are no obvious outliers by plotting some boxplots
TODO: Merge with the disasters